<a href="https://colab.research.google.com/github/ModernOctave/Thermal-Image-Human-Detection/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Dataset

Create file containing all urls of dataset parts.

In [7]:
%%writefile dataset_parts.txt
https://adas-dataset-v2.flirconservator.com/dataset/parts/FLIR_ADAS_v2.zip.00
https://adas-dataset-v2.flirconservator.com/dataset/parts/FLIR_ADAS_v2.zip.01
https://adas-dataset-v2.flirconservator.com/dataset/parts/FLIR_ADAS_v2.zip.02
https://adas-dataset-v2.flirconservator.com/dataset/parts/FLIR_ADAS_v2.zip.03
https://adas-dataset-v2.flirconservator.com/dataset/parts/FLIR_ADAS_v2.zip.04
https://adas-dataset-v2.flirconservator.com/dataset/parts/FLIR_ADAS_v2.zip.05
https://adas-dataset-v2.flirconservator.com/dataset/parts/FLIR_ADAS_v2.zip.06
https://adas-dataset-v2.flirconservator.com/dataset/parts/FLIR_ADAS_v2.zip.07
https://adas-dataset-v2.flirconservator.com/dataset/parts/FLIR_ADAS_v2.zip.08
https://adas-dataset-v2.flirconservator.com/dataset/parts/FLIR_ADAS_v2.zip.09
https://adas-dataset-v2.flirconservator.com/dataset/parts/FLIR_ADAS_v2.zip.10
https://adas-dataset-v2.flirconservator.com/dataset/parts/FLIR_ADAS_v2.zip.11
https://adas-dataset-v2.flirconservator.com/dataset/parts/md5sums_parts.txt

Overwriting dataset_parts.txt


In [11]:
%%writefile dataset_parts.txt
https://adas-dataset-v2.flirconservator.com/dataset/full/FLIR_ADAS_v2.zip

Writing dataset_parts.txt


Download all parts

In [12]:
!cat dataset_parts.txt | xargs -n 1 -P 0 wget

--2022-04-10 09:00:35--  https://adas-dataset-v2.flirconservator.com/dataset/full/FLIR_ADAS_v2.zip
Resolving adas-dataset-v2.flirconservator.com (adas-dataset-v2.flirconservator.com)... 108.156.120.82, 108.156.120.128, 108.156.120.101, ...
Connecting to adas-dataset-v2.flirconservator.com (adas-dataset-v2.flirconservator.com)|108.156.120.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11865443849 (11G) [application/zip]
Saving to: ‘FLIR_ADAS_v2.zip’

FLIR_ADAS_v2.zip    100%[===================>]  11.05G   206MB/s    in 64s     

2022-04-10 09:01:39 (177 MB/s) - ‘FLIR_ADAS_v2.zip’ saved [11865443849/11865443849]



Verify parts checksums

In [9]:
!md5sum -c md5sums_parts.txt

FLIR_ADAS_v2.zip.00: OK
FLIR_ADAS_v2.zip.01: OK
FLIR_ADAS_v2.zip.02: OK
FLIR_ADAS_v2.zip.03: OK
FLIR_ADAS_v2.zip.04: OK
FLIR_ADAS_v2.zip.05: OK
FLIR_ADAS_v2.zip.06: OK
FLIR_ADAS_v2.zip.07: OK
FLIR_ADAS_v2.zip.08: OK
FLIR_ADAS_v2.zip.09: OK
FLIR_ADAS_v2.zip.10: OK
FLIR_ADAS_v2.zip.11: OK


Combine and extract

In [ ]:
!cat FLIR_ADAS_v2.zip.* > FLIR_ADAS_v2.zip

In [13]:
!unzip -q FLIR_ADAS_v2.zip

Remove temp files

In [14]:
!rm FLIR* dataset_parts.txt md5sums_parts.txt

rm: cannot remove 'md5sums_parts.txt': No such file or directory


# Create datasets

In [1]:
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from PIL import Image
import requests
from pycocotools.coco import COCO
import random

In [2]:
dataset_path = "images_thermal_train/"
coco_annotation_file_path = dataset_path+"coco.json"
coco_annotation = COCO(annotation_file=coco_annotation_file_path)

loading annotations into memory...
Done (t=1.88s)
creating index...
index created!


In [3]:
# Get all images
img_ids = coco_annotation.getImgIds()[:1000]
# print(len(img_ids))

# Get category info
cat_ids = coco_annotation.getCatIds()
cat = dict(zip(cat_ids,[cat['name'] for cat in coco_annotation.loadCats(cat_ids)]))
# print(cat)

human = []
no_human = []

for i, img_id in enumerate(img_ids):
  # Open img
  img_info = coco_annotation.loadImgs([img_id])[0]
  img_file_name = img_info["file_name"]
  im = np.array(Image.open(dataset_path+img_file_name))

  # Get all the annotations for the specified image.
  ann_ids = coco_annotation.getAnnIds(imgIds=[img_id], iscrowd=None)
  anns = coco_annotation.loadAnns(ann_ids)

  if 'person' in [cat[ann['category_id']] for ann in anns]:
    human.append(im)
  else:
    no_human.append(im)
  
  if i%(int(len(img_ids)/10)) == 0:
    print(str(int(i/len(img_ids)*100))+"%")



print("Training dataset created!")
# print('person' in [cat[ann['category_id']] for ann in anns])
# im = Image.open('images_thermal_train/'+img_file_name)
# display(im)

0%
10%
20%
30%
40%
50%
60%
70%
80%
90%
Training dataset created!


In [4]:
half_size = min(len(human),len(no_human))
print(half_size)
train_images = human[:half_size]+no_human[:half_size]
train_labels = list(np.ones(half_size))+list(np.zeros(half_size))
print(len(train_labels))
dataset = list(zip(train_images, train_labels))
random.shuffle(dataset)
train_images, train_labels = zip(*dataset)
train_images = (np.expand_dims(train_images, axis=-1)/255.).astype(np.float32)
train_labels = np.array(train_labels).astype(np.int64)
human = None
no_human = None

168
336


In [5]:
print(sum(train_labels)/len(train_labels)*100)

50.0


In [19]:
cat_ids = coco_annotation.getCatIds()
print(f"Number of Unique Categories: {len(cat_ids)}")
print("Category IDs:")
print(cat_ids)  # The IDs are not necessarily consecutive.

# Category IDs.
cat_ids = coco_annotation.getCatIds()
print(f"Number of Unique Categories: {len(cat_ids)}")
print("Category IDs:")
print(cat_ids)  # The IDs are not necessarily consecutive.

# All categories.
cats = coco_annotation.loadCats(cat_ids)
cat_names = [cat["name"] for cat in cats]
print("Categories Names:")
print(cat_names)

# Category ID -> Category Name.
query_id = cat_ids[0]
query_annotation = coco_annotation.loadCats([query_id])[0]
query_name = query_annotation["name"]
query_supercategory = query_annotation["supercategory"]
print("Category ID -> Category Name:")
print(
    f"Category ID: {query_id}, Category Name: {query_name}, Supercategory: {query_supercategory}"
)

# Category Name -> Category ID.
query_name = cat_names[0]
query_id = coco_annotation.getCatIds(catNms=[query_name])[0]
print("Category Name -> ID:")
print(f"Category Name: {query_name}, Category ID: {query_id}")

# Get the ID of all the images containing the object of the category.
img_ids = coco_annotation.getImgIds(catIds=[query_id])
print(f"Number of Images Containing {query_name}: {len(img_ids)}")

# Pick one image.
img_id = img_ids[2]
img_info = coco_annotation.loadImgs([img_id])[0]
img_file_name = img_info["file_name"]
print(img_info)
print(
    f"Image ID: {img_id}, File Name: {img_file_name}"
)

# Get all the annotations for the specified image.
ann_ids = coco_annotation.getAnnIds(imgIds=[img_id], iscrowd=None)
anns = coco_annotation.loadAnns(ann_ids)
print(f"Annotations for Image ID {img_id}:")
print(anns)

# Use URL to load image.
im = Image.open("images_thermal_train/"+img_file_name)

# Save image and its labeled version.
plt.axis("off")
plt.imshow(np.asarray(im))
plt.savefig(f"{img_id}.jpg", bbox_inches="tight", pad_inches=0)
# Plot segmentation and bounding box.
coco_annotation.showAnns(anns, draw_bbox=True)
plt.savefig(f"{img_id}_annotated.jpg", bbox_inches="tight", pad_inches=0)

loading annotations into memory...
Done (t=3.00s)
creating index...
index created!
Number of Unique Categories: 80
Category IDs:
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80]
Number of Unique Categories: 80
Category IDs:
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80]
Categories Names:
['person', 'bike', 'car', 'motor', 'airplane', 'bus', 'train', 'truck', 'boat', 'light', 'hydrant', 'sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'deer', 'sheep', 'cow', 'el

# Model

Install dependencies

In [6]:
# Import Tensorflow 2.0
%tensorflow_version 2.x
import tensorflow as tf 

Create model

In [19]:
print(np.shape(train_images[[0]]))

(1, 512, 640, 1)


In [29]:
def build_cnn_model():
    cnn_model = tf.keras.Sequential([
                                     
        tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation=tf.nn.relu, input_shape=(512, 640, 1)),

        tf.keras.layers.MaxPool2D(pool_size=(2,2)),

        tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation=tf.nn.relu),

        tf.keras.layers.MaxPool2D(pool_size=(2,2)),

        tf.keras.layers.Flatten(),

        tf.keras.layers.Dense(128, activation=tf.nn.relu),

        tf.keras.layers.Dense(128, activation=tf.nn.relu),

        tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
    ])
    
    return cnn_model
  
cnn_model = build_cnn_model()
# Initialize the model by passing some data through
cnn_model.predict(train_images[[0]])
# Print the summary of the layers in the model.
print(cnn_model.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 510, 638, 32)      320       
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 255, 319, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 253, 317, 64)      18496     
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 126, 158, 64)     0         
 g2D)                                                            
                                                                 
 flatten_7 (Flatten)         (None, 1274112)           0         
                                                                 
 dense_18 (Dense)            (None, 128)              

Train model

In [30]:
cnn_model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [33]:
cnn_model.fit(train_images, train_labels, epochs=5)

Epoch 1/5


ResourceExhaustedError: ignored